# Feature Extraction

In [1]:
# Importing libraries
import pandas as pd

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

In [2]:
# We shall load and inspect the dataset
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# We shall inspect the description of the dataset's features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# We shall calculate some basic statistics regarding the dataset
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# These columns should be redundant, so We'll drop them
df.drop(columns=["PassengerId", "Name", "Ticket"], inplace=True)

## Handling Missing Data

In [6]:
# We first check whether there is any missing data in the dataset
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [7]:
# Considering the "Cabin" column has a lot of missing data and there is no good 
# way to fill it, We'll drop it.
df.drop(columns=["Cabin"], inplace=True)

In [8]:
# For the missing data in the column "Embarked", We'll fill the missing data using 
# the column's mode.
df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)

In [9]:
# For the missing data in the column "Age", We'll fill the missing data using 
# the column's median according to the entry's sex and passenger class. This 
# is done due to my suspicion that "Age" highly correlates with a passenger's 
# survival chances.
df["Age"] = df.groupby(["Sex", "Pclass"], group_keys=False)["Age"].apply(lambda entry: entry.fillna(entry.median()))

In [10]:
# We shall inspect the result of the imputation
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
# We shall verify that there is no more missing data in the dataset
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

## Label Encoding

In [12]:
# We shall numerically encode the "Sex" feature
df["Sex"] = df["Sex"].map( {"female": 1, "male": 0} ).astype(int)

# We shall also numerically encode the "Embarked" feature
df["Embarked"] = df["Embarked"].map( {"S" : 0, "C" : 1, "Q" : 2}).astype(int)

In [13]:
# We shall inspect the result of the label encoding
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


## Feature Extraction & Model Building

In [14]:
# We shall appropriately select the features to be used for the model and the 
# target label.
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [15]:
# We shall inspect the selected features before proceeding
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,0
887,1,1,19.0,0,0,30.0000,0
888,3,1,21.5,1,2,23.4500,0
889,1,0,26.0,0,0,30.0000,1


In [16]:
# We shall inspect the target label before proceeding
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [17]:
# To train the model, we shall first split the available data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
# To make the non-LDA model, we shall use the following pipeline:
# MinMaxScaler  -> LogisticRegression
# The MinMaxScaler is needed to ensure the higher value features wouldn't 
# influence the model much more that the other features.
# The LogisticRegression is used as the classifier for the model.
pipe = make_pipeline(MinMaxScaler(), LogisticRegression())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.8324022346368715

In [19]:
# To make the LDA model, we shall use the following pipeline:
# MinMaxScaler -> LinearDiscriminantAnalysis -> LogisticRegression
# The MinMaxScaler is needed to ensure the higher value features wouldn't 
# influence the model much more that the other features.
# The LinearDiscriminantAnalysis is used to project the dataset onto a 
# lower-dimensional space, extracting features.
# The LogisticRegression is used as the classifier for the model.
lda_pipe = make_pipeline(MinMaxScaler(), LinearDiscriminantAnalysis(), LogisticRegression())
lda_pipe.fit(X_train, y_train)
lda_pipe.score(X_test, y_test)

0.8379888268156425